In [14]:
import pandas as pd
import zipfile
import os


In [15]:
# Convert .pbit to .zip
pbit_file = 'Multilingual Dashboard_TestApp.pbit'
zip_file = 'Multilingual Dashboard_TestApp.zip'
os.rename(pbit_file, zip_file)

# Extract .zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    extract_folder = 'Multilingual Dashboard_TestApp Extracted'
    zip_ref.extractall(extract_folder)





In [16]:
#again renaming just to avoid loosing app
pbit_file = 'Multilingual Dashboard_TestApp.pbit'
zip_file = 'Multilingual Dashboard_TestApp.zip'
os.rename(zip_file,pbit_file)

In [17]:
# Read Layout file
import json
filepath = r"Multilingual Dashboard_TestApp Extracted\Report\Layout"
with open(filepath, "r", encoding = 'utf-16 le') as f:
    data = json.load(f)    

In [18]:
excel_file = 'Label_output.xlsx'
#storing objects title sheet wise
with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    combine = []
    for i in data["sections"]:
            currentsheet = i["displayName"]
            
            for j in i["visualContainers"]:
                content = json.loads(j["config"])              
                try:
                    objectid = content["name"]
                    objectlabel = content["singleVisual"]["vcObjects"]["title"][0]["properties"]["text"]["expr"]["Literal"]["Value"]
                    objectlabel = objectlabel[1:-1]
                    combine.append([objectid,objectlabel,objectlabel+"_Updated"])
                    df = pd.DataFrame(combine,columns=["ID","Label","New Label"])
                    df.to_excel(writer, sheet_name=currentsheet, index=False) 
                except:   
                    pass
            combine = []        
                

In [19]:
#Extracting labels for tables

excel_file_path = 'TableLableExtracted.xlsx'

with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    global df, olddf,newdf
    previoussheet = ""
    for i in data["sections"]:
        currentsheet = i["displayName"]
        for j in i["visualContainers"]:
            content = json.loads(j["config"])
            if(content["singleVisual"]["visualType"]=="tableEx"):
                # print(content["singleVisual"])
                combine = []
                exceldf = pd.read_excel(excel_file,sheet_name=currentsheet)
                
                for key,value in content["singleVisual"]["columnProperties"].items():
                    try:
                        objectcolumnId = key + content["name"]+value["displayName"]
                        oldlabel = value["displayName"]
                        combine.append([objectcolumnId,oldlabel,oldlabel+"_Updated"])
                    except:
                        pass     
                if(previoussheet == currentsheet ):
                    start_row = len(olddf)
                    df = pd.DataFrame(combine,columns=["ID","Label","New Label"])
                    newdf = pd.concat([olddf, df],ignore_index = True)
                    newdf.to_excel(writer, sheet_name=currentsheet, index=False)
                    olddf = newdf.copy()
                    
                else:
                    df = pd.DataFrame(combine,columns=["ID","Label","New Label"])
                    df.to_excel(writer, sheet_name=currentsheet, index=False)
                    olddf = df.copy()
                
                
                previoussheet = currentsheet


In [20]:
#Extracting labels for all type of barcharts and pie charts

excel_file_path = 'BarchartLableExtracted.xlsx'

with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    global df, olddf,newdf
    previoussheet = ""
    for i in data["sections"]:
        currentsheet = i["displayName"]
        for j in i["visualContainers"]:
            content = json.loads(j["config"])
            if(content["singleVisual"]["visualType"] in ["clusteredColumnChart","clusteredBarChart","barChart","donutChart","columnChart"]):
                combine = []
                exceldf = pd.read_excel(excel_file,sheet_name=currentsheet)
                
                for key,value in content["singleVisual"]["columnProperties"].items():

                    try:
                        objectcolumnId = key + content["name"]+value["displayName"]
                        # print(content)
                        # print(currentsheet,value["displayName"])
                        oldlabel = value["displayName"]
                        combine.append([objectcolumnId,oldlabel,oldlabel+"_Updated"])
                    except:
                        pass   

                if(previoussheet == currentsheet):
                    start_row = len(olddf)
                    df = pd.DataFrame(combine,columns=["ID","Label","New Label"])
                    newdf = pd.concat([olddf, df],ignore_index = True)
                    newdf.to_excel(writer, sheet_name=currentsheet, index=False)
                    olddf = newdf.copy()
                else:
                    df = pd.DataFrame(combine,columns=["ID","Label","New Label"])
                    df.to_excel(writer, sheet_name=currentsheet, index=False)
                    olddf = df.copy()  
                

                previoussheet = currentsheet